In [2]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import nltk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

In [5]:
raw_filtered_data = pd.read_csv("spam.csv", encoding='latin1')

In [7]:
raw_filtered_data.drop({"Unnamed: 2","Unnamed: 3","Unnamed: 4"},axis = 1,inplace = True)

In [10]:
raw_filtered_data

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [11]:
def make_label(x):
    if x=="spam":
        return 1
    else: 
        return 0

In [12]:
raw_filtered_data['v1'] = raw_filtered_data['v1'].map(make_label)
raw_filtered_data

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [13]:
print(f"Shape Before: {raw_filtered_data.shape}")
final_filtered_data = raw_filtered_data.drop_duplicates(subset=["v2"], keep='first', inplace=False)
print(f"Shape After:  {final_filtered_data.shape}")

Shape Before: (5572, 2)
Shape After:  (5169, 2)


In [15]:
count_vect = CountVectorizer() # To compute Bag of Words representation(from scikit-learn) 
final_counts = count_vect.fit_transform(final_filtered_data['v2'].values)

In [16]:
import re
import string 
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [18]:
stop_words = {'a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves'}
snow = nltk.stem.SnowballStemmer('english')

In [19]:
def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,' ',sentence)
    return cleantext

def clean_punctuation(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned

In [59]:
i=0
all_spam_words = []
all_normal_words = []
final_strings = []
s = ''
for sent in final_filtered_data['v2']:
    filtered_sentence = []
    sent= cleanhtml(sent)
    for w in sent.split():
        cleaned_word = clean_punctuation(w)
        if((cleaned_word.isalpha()) & (len(cleaned_word)>2)):
            s = (snow.stem(cleaned_word.lower())).encode('utf8')
            filtered_sentence.append(s)
            if(final_filtered_data['v1'].values)[i] == 1:
                all_spam_words.append(s)
            else:
                all_normal_words.append(s)
    strl = b" ".join(filtered_sentence) 
    final_strings.append(strl)  
    i+=1

In [60]:
final_filtered_data = final_filtered_data.copy()
final_filtered_data['CleanedText'] = final_strings

In [61]:
final_filtered_data

,index,v1,v2,CleanedText
0,0,0,"Go until jurong point, crazy.. Available only ...",b'until jurong avail onli bugi great world cin...
1,1,0,Ok lar... Joking wif u oni...,b'joke wif'
2,2,1,Free entry in 2 a wkly comp to win FA Cup fina...,b'free entri wkli comp win cup final tkts may ...
3,3,0,U dun say so early hor... U c already then say...,b'dun say earli alreadi then'
4,4,0,"Nah I don't think he goes to usf, he lives aro...",b'nah dont think goe live around here though'
...,...,...,...,...
5164,5567,1,This is the 2nd time we have tried 2 contact u...,b'this the time have tri contact have won the ...
5165,5568,0,Will Ì_ b going to esplanade fr home?,b'will go esplanad home'
5166,5569,0,"Pity, * was in mood for that. So...any other s...",b'was mood for other suggest'
5167,5570,0,The guy did some bitching but I acted like i'd...,b'the guy did some bitch but act like interest...


In [62]:
conn = sqlite3.connect('final.sqlite')
c = conn.cursor()
conn.text_factory = str
final_filtered_data.to_sql('Messages', conn,schema = None, if_exists = 'replace')

OperationalError: table "Messages" already exists

In [ ]:
conn = sqlite3.connect('final.sqlite')
final_filtered_data = pd.read_sql_query("""
SELECT *
FROM messages
""",conn)

In [63]:
pd.read_sql_query("""
SELECT count(*)
FROM messages
WHERE v1 == 1
""",conn)

,count(*)
0,653


In [64]:
pd.read_sql_query("""
SELECT count(*)
FROM messages
WHERE v1 == 0
""",conn)

,count(*)
0,4516


In [65]:
all_normal_words[::3000]

[b'until',
 b'think',
 b'gift',
 b'make',
 b'you',
 b'ask',
 b'tonight',
 b'with',
 b'and',
 b'come',
 b'well',
 b'and',
 b'beeen',
 b'you']

In [67]:
all_spam_words[::1000]

[b'free',
 b'repli',
 b'stori',
 b'txt',
 b'tone',
 b'with',
 b'are',
 b'urgent',
 b'must',
 b'mnths']

In [69]:
freq_dist_positive = nltk.FreqDist(all_spam_words)
freq_dist_negative = nltk.FreqDist(all_normal_words)
freq_dist_positive

FreqDist({b'call': 313, b'you': 249, b'your': 249, b'for': 182, b'the': 181, b'free': 174, b'now': 134, b'text': 117, b'have': 115, b'txt': 114, ...})

In [70]:
print("Most common positive words: ",freq_dist_positive.most_common()[:10])
print("Most common negative words: ",freq_dist_negative.most_common()[:10])

Most common positive words:  [(b'call', 313), (b'you', 249), (b'your', 249), (b'for', 182), (b'the', 181), (b'free', 174), (b'now', 134), (b'text', 117), (b'have', 115), (b'txt', 114)]
Most common negative words:  [(b'you', 1651), (b'the', 1043), (b'and', 815), (b'that', 529), (b'for', 474), (b'have', 456), (b'your', 433), (b'but', 396), (b'are', 388), (b'not', 363)]


# TF-IDF

In [71]:
tf_idf_vect = TfidfVectorizer(ngram_range = (1,2))
final_tf_idf = tf_idf_vect.fit_transform(final_filtered_data['CleanedText'].values)

In [72]:
final_tf_idf

<5169x33552 sparse matrix of type '<class 'numpy.float64'>'
	with 87814 stored elements in Compressed Sparse Row format>

In [73]:
features = tf_idf_vect.get_feature_names_out()
len(features)

33552

In [74]:
features[100000:100010]

array([], dtype=object)

In [75]:
print(len(final_tf_idf[3,:].toarray()[0]))

33552


In [76]:
def top_tfidf_features(row,features,top_n=25):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

top_tfidf = top_tfidf_features(final_tf_idf[1,:].toarray()[0], features,25)

In [77]:
top_tfidf

,feature,tfidf
0,joke wif,0.679115
1,joke,0.555717
2,wif,0.479563
3,got free,0.000000
4,got have,0.000000
5,got got,0.000000
6,got from,0.000000
7,got fring,0.000000
8,got friend,0.000000
9,ìïll take,0.000000


In [79]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from collections import Counter
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [96]:
y_final = final_filtered_data['v1']

In [97]:
x_train, x_test, y_train, y_test = train_test_split(final_tf_idf,y_final,train_size = 0.7,random_state = 42) 

### XGBoost

In [103]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate

In [99]:
model = XGBClassifier()

In [100]:
model.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [101]:
predictions = model.predict(x_test)

In [102]:
accuracy = accuracy_score(predictions,y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9606705351386202
